# Computation of Scalar Products

By now, we are able to construct orthogonal polynomials, and to construct quadrature rules for a given nonnegative weight function, respectively.
Now we combine both ideas to solve integrals involving the orthogonal polynomials
$$
\langle \phi_{i_1} \phi_{i_2} \cdots \phi_{i_{m-1}}, \phi_{i_m} \rangle
= \int \phi_{i_1}(t) \phi_{i_2}(t) \cdots \phi_{i_{m-1}}(t) \phi_{i_m}(t) w(t) \mathrm{d} t,
$$
both for the univariate and multivariate case.
The integrand is a polynomial (possibly multivariate) that can be solved exactly with the appropriate Gauss quadrature rules.

Notice: To simplify notation we drop the integration interval.
It is clear from the context.


## Univariate Polynomials
### Classical Polynomials
Let's begin with a univariate basis for some *classical* orthogonal polynomial

In [1]:
using PolyChaos
deg, n = 4, 20
s_α, s_β = 2.1, 3.2
opq = Beta01OrthoPoly(deg,s_α,s_β;Nrec=n)

Beta01OrthoPoly(4, [0.396226, 0.453089, 0.473266, 0.482729, 0.487923, 0.491081, 0.493143, 0.494564, 0.495585, 0.496343, 0.496921, 0.497372, 0.497731, 0.498021, 0.498259, 0.498456, 0.498622, 0.498762, 0.498882, 0.498985], [1.0, 0.0379732, 0.0495282, 0.0544964, 0.0570749, 0.0585822, 0.0595388, 0.0601835, 0.0606385, 0.0609715, 0.0612226, 0.0614165, 0.0615694, 0.0616921, 0.061792, 0.0618745, 0.0619433, 0.0620014, 0.0620509, 0.0620933], Beta01Measure(getfield(PolyChaos, Symbol("##108#109")){Float64,Float64}(2.1, 3.2), (0, 1), false, 2.1, 3.2), Quad("golubwelsch", 19, [0.00878891, 0.0284501, 0.0585213, 0.0983381, 0.147024, 0.203505, 0.266539, 0.334737, 0.406597, 0.480536, 0.554927, 0.62813, 0.698535, 0.76459, 0.824841, 0.877962, 0.922785, 0.958329, 0.983839], [0.00115168, 0.00700726, 0.0203149, 0.0414026, 0.0677717, 0.0947199, 0.116731, 0.129136, 0.129476, 0.118122, 0.0980077, 0.0735996, 0.0495353, 0.0293896, 0.014964, 0.00625271, 0.00197967, 0.000403515, 3.42927e-5]))

By default, an $n$-point Gauss quadrature rule is create relative to the underlying measure `op.measure`, where $n$ is the number of recurrence coefficients stored in `op.α` and `op.β`.

To compute the squared norms
$$
\| \phi_k \|^2 = \langle \phi_k, \phi_k  \rangle
= \int \phi_k(t) \phi_k(t) w(t) \mathrm{d} t
$$

of the basis we call `computeSP2()`

In [2]:
normsq = computeSP2(opq)

5-element Array{Float64,1}:
 1.0                   
 0.03797318144060756   
 0.0018807430768498865 
 0.00010249372143217383
 5.849823409553853e-6  

For the general case
$$
\langle \phi_{i_1} \phi_{i_2} \cdots \phi_{i_{m-1}}, \phi_{i_m} \rangle
= \int \phi_{i_1}(t) \phi_{i_2}(t) \cdots \phi_{i_{m-1}}(t) \phi_{i_m}(t) w(t) \mathrm{d} t,
$$
there exists a type `Tensor` that requires only two arguments: the *dimension* $m \geq 1$, and an `OrthoPolyQ`

In [3]:
m = 3
t = Tensor(3,opq)

Tensor(3,   [1 ]  =  1.0
  [6 ]  =  0.0379732
  [11]  =  0.00188074
  [16]  =  0.000102494
  [21]  =  5.84982e-6
  [22]  =  0.00215924
  [23]  =  0.00188074
  [27]  =  0.000144891
  [28]  =  0.000102494
  [32]  =  8.86598e-6
        ⋮
  [37]  =  5.36411e-7
  [43]  =  0.000127149
  [44]  =  1.0934e-5
  [45]  =  5.84982e-6
  [48]  =  7.80614e-6
  [49]  =  7.09802e-7
  [53]  =  4.73123e-7
  [64]  =  9.40423e-7
  [65]  =  5.08904e-7
  [69]  =  6.53232e-8
  [85]  =  3.55404e-8, getfield(PolyChaos, Symbol("#getfun#41")){Int64,Beta01OrthoPoly,SparseArrays.SparseVector{Float64,Int64}}(3, Beta01OrthoPoly(4, [0.396226, 0.453089, 0.473266, 0.482729, 0.487923, 0.491081, 0.493143, 0.494564, 0.495585, 0.496343, 0.496921, 0.497372, 0.497731, 0.498021, 0.498259, 0.498456, 0.498622, 0.498762, 0.498882, 0.498985], [1.0, 0.0379732, 0.0495282, 0.0544964, 0.0570749, 0.0585822, 0.0595388, 0.0601835, 0.0606385, 0.0609715, 0.0612226, 0.0614165, 0.0615694, 0.0616921, 0.061792, 0.0618745, 0.0619433, 0.0620014, 

To get the desired entries, `Tensor`comes with a `get()` function that is called for some index $a \in \mathbb{N}_0^m$ that has the entries $a = [i_1, i_2, \dots, i_m]$.
For example


In [4]:
t.get([1,2,3])

0.00010249372143217367

Or using comprehension

In [5]:
T = [ t.get([i1,i2,i3]) for i1=0:dim(opq)-1,i2=0:dim(opq)-1,i3=0:dim(opq)-1]

5×5×5 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0        0.0         0.0          0.0       
 0.0  0.0379732  0.0         0.0          0.0       
 0.0  0.0        0.00188074  0.0          0.0       
 0.0  0.0        0.0         0.000102494  0.0       
 0.0  0.0        0.0         0.0          5.84982e-6

[:, :, 2] =
 0.0        0.0379732   0.0          0.0          0.0       
 0.0379732  0.00215924  0.00188074   0.0          0.0       
 0.0        0.00188074  0.000144891  0.000102494  0.0       
 0.0        0.0         0.000102494  8.86598e-6   5.84982e-6
 0.0        0.0         0.0          5.84982e-6   5.36411e-7

[:, :, 3] =
 0.0         0.0          0.00188074   0.0          0.0       
 0.0         0.00188074   0.000144891  0.000102494  0.0       
 0.00188074  0.000144891  0.000127149  1.0934e-5    5.84982e-6
 0.0         0.000102494  1.0934e-5    7.80614e-6   7.09802e-7
 0.0         0.0          5.84982e-6   7.09802e-7   4.73123e-7

[:, :, 4] =
 0.0          0.0          0.0          

Notice that we can cross-check the results.

In [6]:
using LinearAlgebra
@show normsq == LinearAlgebra.diag(T[:,:,1])
@show normsq == LinearAlgebra.diag(T[:,1,:])
@show normsq == LinearAlgebra.diag(T[1,:,:])

normsq == LinearAlgebra.diag(T[:, :, 1]) = true
normsq == LinearAlgebra.diag(T[:, 1, :]) = true
normsq == LinearAlgebra.diag(T[1, :, :]) = true


true

Also, `normsq` can be computed analogously in `Tensor` format

In [7]:
t2 = Tensor(2,opq)
@show normsq == [ t2.get([i,i]) for i=0:dim(opq)-1]

normsq == [t2.get([i, i]) for i = 0:dim(opq) - 1] = true


true

### Arbitrary Weights
Of course, the type `OrthoPolyQ` can be constructed for arbitrary weights $w(t)$.
In this case we have to compute the orthogonal basis and the respective quadrature rule.
Let's re-work the above example by hand.

In [15]:
using SpecialFunctions
supp = (0,1)
function w(t)
    supp[1]<=t<=supp[2] ? (t^(s_α-1)*(1-t)^(s_β-1)/SpecialFunctions.beta(s_α,s_β)) : error("$t not in support")
end
my_meas = Measure("my_meas",w,supp,false,Dict())
my_opq = OrthoPoly("my_op",deg,my_meas;Nrec=n)

OrthoPoly("my_op", 4, [0.396226, 0.453089, 0.473266, 0.482729, 0.487923, 0.491081, 0.493143, 0.494564, 0.495585, 0.496343, 0.496921, 0.497372, 0.497731, 0.498021, 0.498259, 0.498456, 0.498622, 0.498762, 0.498882, 0.498985], [1.0, 0.0379732, 0.0495282, 0.0544964, 0.0570749, 0.0585822, 0.0595388, 0.0601835, 0.0606385, 0.0609715, 0.0612226, 0.0614165, 0.0615694, 0.0616921, 0.061792, 0.0618745, 0.0619433, 0.0620014, 0.0620509, 0.0620933], Measure("my_meas", w, (0, 1), false, Dict{Any,Any}()), Quad("golubwelsch", 19, [0.00878891, 0.0284501, 0.0585213, 0.0983381, 0.147024, 0.203505, 0.266539, 0.334737, 0.406597, 0.480536, 0.554927, 0.62813, 0.698535, 0.76459, 0.824841, 0.877962, 0.922785, 0.958329, 0.983839], [0.00115168, 0.00700726, 0.0203149, 0.0414026, 0.0677717, 0.0947199, 0.116731, 0.129136, 0.129476, 0.118122, 0.0980077, 0.0735996, 0.0495353, 0.0293896, 0.014964, 0.00625271, 0.00197967, 0.000403515, 3.42927e-5]))

Now we can compute the squared norms $\| \phi_k \|^2$

In [16]:
my_normsq = computeSP2(my_opq)

5-element Array{Float64,1}:
 0.9999999999996537   
 0.037973181440551666 
 0.0018807430768424916
 0.0001024937214313095
 5.849823409460972e-6 

And the tensor

In [17]:
my_t = Tensor(m,my_opq)
my_T = [ my_t.get([i1,i2,i3]) for i1=0:dim(opq)-1,i2=0:dim(opq)-1,i3=0:dim(opq)-1]

5×5×5 Array{Float64,3}:
[:, :, 1] =
 1.0  0.0        0.0         0.0          0.0       
 0.0  0.0379732  0.0         0.0          0.0       
 0.0  0.0        0.00188074  0.0          0.0       
 0.0  0.0        0.0         0.000102494  0.0       
 0.0  0.0        0.0         0.0          5.84982e-6

[:, :, 2] =
 0.0        0.0379732   0.0          0.0          0.0       
 0.0379732  0.00215924  0.00188074   0.0          0.0       
 0.0        0.00188074  0.000144891  0.000102494  0.0       
 0.0        0.0         0.000102494  8.86598e-6   5.84982e-6
 0.0        0.0         0.0          5.84982e-6   5.36411e-7

[:, :, 3] =
 0.0         0.0          0.00188074   0.0          0.0       
 0.0         0.00188074   0.000144891  0.000102494  0.0       
 0.00188074  0.000144891  0.000127149  1.0934e-5    5.84982e-6
 0.0         0.000102494  1.0934e-5    7.80614e-6   7.09802e-7
 0.0         0.0          5.84982e-6   7.09802e-7   4.73123e-7

[:, :, 4] =
 0.0          0.0          0.0          

Let's compare the results:

In [18]:
@show abs.(normsq-my_normsq)
@show norm(T-my_T)

abs.(normsq - my_normsq) = [3.46279e-13, 5.58928e-14, 7.39491e-15, 8.64326e-16, 9.28806e-17]
norm(T - my_T) = 3.600950231679057e-13


3.600950231679057e-13

Notice: __The possibility to create quadrature rules for arbitrary weights should be reserved to cases different from *classical* ones.__

### Multivariate Polynomials
For multivariate polynomials the syntax for `Tensor` is very much alike, except that we are dealing with the type `MultiOrthoPoly` now.

In [19]:
mop = MultiOrthoPoly([opq,my_opq],deg)

MultiOrthoPoly(["Beta01OrthoPoly", "my_op"], 4, 15, [0 0; 1 0; … ; 1 3; 0 4], ProductMeasure(getfield(PolyChaos, Symbol("#w#36")){Array{AbstractOrthoPoly,1}}(AbstractOrthoPoly[Beta01OrthoPoly(4, [0.396226, 0.453089, 0.473266, 0.482729, 0.487923, 0.491081, 0.493143, 0.494564, 0.495585, 0.496343, 0.496921, 0.497372, 0.497731, 0.498021, 0.498259, 0.498456, 0.498622, 0.498762, 0.498882, 0.498985], [1.0, 0.0379732, 0.0495282, 0.0544964, 0.0570749, 0.0585822, 0.0595388, 0.0601835, 0.0606385, 0.0609715, 0.0612226, 0.0614165, 0.0615694, 0.0616921, 0.061792, 0.0618745, 0.0619433, 0.0620014, 0.0620509, 0.0620933], Beta01Measure(##108#109{Float64,Float64}(2.1, 3.2), (0, 1), false, 2.1, 3.2), Quad("golubwelsch", 19, [0.00878891, 0.0284501, 0.0585213, 0.0983381, 0.147024, 0.203505, 0.266539, 0.334737, 0.406597, 0.480536, 0.554927, 0.62813, 0.698535, 0.76459, 0.824841, 0.877962, 0.922785, 0.958329, 0.983839], [0.00115168, 0.00700726, 0.0203149, 0.0414026, 0.0677717, 0.0947199, 0.116731, 0.129136, 0.

In [20]:
mt2 = Tensor(2,mop)
mt3 = Tensor(3,mop)
mT2 = [ mt2.get([i,i]) for i=0:dim(mop)-1 ]

15-element Array{Float64,1}:
 0.9999999999996537    
 0.03797318144059441   
 0.037973181440551666  
 0.0018807430768492354 
 0.00144196250871918   
 0.0018807430768424916 
 0.00010249372143213834
 7.141779810028215e-5  
 7.141779810010645e-5  
 0.0001024937214313095 
 5.8498234095518276e-6 
 3.892012680461295e-6  
 3.5371945211048704e-6 
 3.892012680434203e-6  
 5.849823409460972e-6  

Notice that `mT2` carries the elements of the 2-dimensional tensors for the univariate bases `opq` and `my_opq`.
The encoding is given by the multi-index `mop.ind`

In [21]:
mop.ind

15×2 Array{Int64,2}:
 0  0
 1  0
 0  1
 2  0
 1  1
 0  2
 3  0
 2  1
 1  2
 0  3
 4  0
 3  1
 2  2
 1  3
 0  4

To cross-check the results we can distribute the multi-index back to its univariate indices with the help of `findUnivariateIndices`.

In [22]:
ind_opq = findUnivariateIndices(1,mop.ind)
ind_my_opq = findUnivariateIndices(2,mop.ind)

5-element Array{Int64,1}:
  1
  3
  6
 10
 15

In [23]:
@show mT2[ind_opq] - normsq
@show mT2[ind_my_opq] - my_normsq;

mT2[ind_opq] - normsq = [-3.46279e-13, -1.31492e-14, -6.51172e-16, -3.54941e-17, -2.02526e-18]
mT2[ind_my_opq] - my_normsq = [0.0, 0.0, 0.0, 0.0, 0.0]
